In [ ]:
#Plotting tools
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from numpy.random import multivariate_normal
import matplotlib.ticker as mtick
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
from mpl_toolkits.mplot3d import Axes3D
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors.kde import KernelDensity
# import random
from tqdm import tqdm

import numpy as np
from autotune import TuningProblem
from autotune.space import *
import os, sys, time, json, math
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import csv
from csv import writer
from csv import reader
from sklearn import preprocessing
# HERE = os.path.dirname(os.path.abspath(__file__))
# sys.path.insert(1, os.path.dirname(HERE)+ '/plopper')
# from plopper import Plopper

cs = CS.ConfigurationSpace(seed=1234)
# number of threads
p0= CSH.OrdinalHyperparameter(name='p0', sequence=['2','3','4','5','6','7','8'], default_value='8')
#block size for openmp dynamic schedule
p1= CSH.OrdinalHyperparameter(name='p1', sequence=['10','20','40','64','80','100','128','160','200'], default_value='100')
#clang unrolling
p2= CSH.CategoricalHyperparameter(name='p2', choices=["#pragma clang loop unrolling full", " "], default_value=' ')
#omp parallel
p3= CSH.CategoricalHyperparameter(name='p3', choices=["#pragma omp parallel for", " "], default_value=' ')
# tile size for one dimension for 2D tiling
p4= CSH.OrdinalHyperparameter(name='p4', sequence=['2','4','8','16','32','64','96','128','256'], default_value='96')
# tile size for another dimension for 2D tiling
p5= CSH.OrdinalHyperparameter(name='p5', sequence=['2','4','8','16','32','64','96','128','256'], default_value='256')
# omp placement
p6= CSH.CategoricalHyperparameter(name='p6', choices=['cores','threads','sockets'], default_value='cores')
p7= CSH.CategoricalHyperparameter(name='p7', choices=['compact','scatter','balanced','none','disabled', 'explicit'], default_value='none')

cs.add_hyperparameters([p0, p1, p2, p3, p4, p5, p6, p7])

In [ ]:

# min_max_scaler = preprocessing.MinMaxScaler()
# X_train_minmax = min_max_scaler.fit_transform(X_train)

In [ ]:
N_infer = 10


In [ ]:
param_names = cs.get_hyperparameter_names()
param_vals = [] 
param_obj = {}
for i, p_name in enumerate(param_names): #cs.get_hyperparameter_names()
#     print (i,p_name)
    try: # ordinal 
        vals = list(map(int,list(cs.get_hyperparameters()[i].sequence)))
        param_vals.append(vals)
        X = np.array(vals) 
        X = X[:,np.newaxis]
        transformer = preprocessing.MinMaxScaler().fit(X)
        param_obj[p_name] = transformer
    except: ## norminal 
        vals = list(cs.get_hyperparameters()[i].choices)
#         print (vals)
        param_vals.append(vals)
        X = np.array(vals) 
        X = X[:,np.newaxis]
        transformer = preprocessing.OneHotEncoder(drop='if_binary',sparse=False).fit(X)
        param_obj[p_name] = transformer
## add input
input_sizes = {}
input_sizes['s']  = [100000]  
input_sizes['m']  = [1000000]  
input_sizes['l']  = [5000000]
input_sizes['sm'] = [500000]
input_sizes['ml'] = [2500000]
input_sizes['xl'] = [10000000]
## add inputs sizes 
vals = [input_sizes['s'][0], input_sizes['xl'][0]]
param_vals.append(vals)
X = np.array(vals) 
X = X[:,np.newaxis]
transformer = preprocessing.MinMaxScaler().fit(X)
param_obj['input'] = transformer

In [ ]:
decode = param_obj['input'].inverse_transform(np.array(np.array([[0.5],[0.5]])))
te = decode #np.array([[' '],[' '],['#pragma omp parallel for']])
coded = param_obj['input'].transform(te)
decode = param_obj['p3'].inverse_transform(np.array([[1],[0]]))
te = np.array([[' '],[' '],['#pragma omp parallel for']])
coded = param_obj['p3'].transform(te)
decode = param_obj['p6'].inverse_transform(np.array([[1,0,0]]))
te = np.array([['cores'],['threads'],['sockets']])
coded = param_obj['p6'].transform(te)

In [ ]:
coded_te = param_obj['p6'].inverse_transform(coded)

In [ ]:
coded_te.shape

In [ ]:
# #### selet by best top x%
if False:
    take_n = int(len(y_eval) * 0.1)
    take_idx = np.argsort(y_eval)[-take_n:]
    X_opt = X_eval[take_idx]
    print (X_opt)   
else:
    X_opt = []
    cutoff_p = 0.1

    '''
    #### problem       S       L         XL       XXL
    size (s)      :    12      355       355      355        - nuclides
    gridpoints (g):  11,303   11,303   238,847   501,578     - grid points per nuclide
    particle   (p): 500,000  500,000   500,000   500,000     -  
    lookup     (l):                                          - 
    '''
    n_param = len(param_names)
    for i_size, o3p_tmp in zip(['s','m','l'],[0.297755, 3.00738, 15.0962]):#['s','m','l']: 0.00106, 0.0266395, 3.972039
        dataframe = pd.read_csv("results_rf_"+str(i_size)+"_xsbench.csv") # PROBLEM_SIZE	BLOCK_SIZE	exe_time	LOG(exe_time)	speedup	elapsed_sec 
        array = dataframe.values
        X_eval = array[:,:n_param]#.astype(float) ## ### size, n1, n2, p1,p2,p3,p4,p5,exe_time,speedup       
#         X_eval_encode = np.copy(X_eval)
        X_eval = np.c_[X_eval, np.array([input_sizes[i_size]]*len(X_eval))]
        X_eval_encode = []
        for i, p_name in enumerate(param_obj.keys()):
            x_tmp = X_eval[:,i]
            x_tmp = x_tmp[:,np.newaxis]
            X_eval_encode.append(param_obj[p_name].transform(x_tmp)) ## [0,1]
        X_eval_en = X_eval_encode[0] 
        for i in range(1,len(X_eval_encode)):    
            X_eval_en = np.c_[X_eval_en,X_eval_encode[i]]
                
        y_eval = o3p_tmp / array[:,n_param].astype(float) ## speedup 
        take_n = round(len(y_eval) * cutoff_p)
        take_idx = np.argsort(y_eval)[-take_n:]
        X_opt.extend(X_eval_en[take_idx].astype(float))
    print (X_opt) 

In [ ]:
X_opt_norm = X_opt

colors = ['navy']#, 'cornflowerblue']#, 'darkorange']
kernels = ['gaussian']#, 'tophat']#, 'epanechnikov']

grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(0.001, 10, 10000)},
                    cv=max(2,round(len(X_opt_norm)*0.2))) # 20-fold cross-validation

grid.fit(X_opt_norm)
print ('grid.best_params_',grid.best_params_)

bandwidth_size = grid.best_params_['bandwidth']

In [ ]:
X_plot_cs = cs.sample_configuration(N_infer) 

In [ ]:
X_plot = [] 
for c in X_plot_cs:
    value = list(dict(c).values())
    X_plot.append(value)

In [ ]:
X_plot_0 = []
X_plot = np.c_[X_plot, np.array([input_sizes['xl']]*len(X_plot))]
for i, p_name in enumerate(param_obj.keys()):
    x_tmp = X_plot[:,i]
    x_tmp = x_tmp[:,np.newaxis]
    X_plot_0.append(param_obj[p_name].transform(x_tmp)) ## [0,1]

In [ ]:
X_plot_norm = X_plot_0[0] 
for i in range(1,len(X_plot_0)):    
    X_plot_norm = np.c_[X_plot_norm,X_plot_0[i]]

In [ ]:
lw = 2
for color, kernel in zip(colors, kernels):
    kde = KernelDensity(kernel=kernel, bandwidth=bandwidth_size).fit(X_opt_norm)
    X_te = []
    for i in range(len(X_plot_norm)):
            log_den = kde.score_samples(np.array([X_plot_norm[i]]))
            X_te.append([X_plot[i],np.exp(log_den[0])])

In [ ]:
X_test = np.copy(X_te)
take_n_te = N_infer
take_idx = np.argsort(-1*np.array(X_test)[:,1])[:take_n_te]
X_opt_te = []# np.array(X_test)[take_idx]
for idx in take_idx:
    x_sample = X_test[idx]
    X_opt_te.append([x_sample[0],x_sample[1]])

In [ ]:
new_kde = X_opt_te

In [ ]:
HERE = os.path.dirname(os.path.abspath(__file__))
sys.path.insert(1, os.path.dirname(HERE)+ '/plopper')
from plopper import Plopper


dir_path = os.path.dirname(os.path.realpath(__file__))
kernel_idx = dir_path.rfind('/')
kernel = dir_path[kernel_idx+1:]
obj = Plopper(dir_path+'/mmp.c',dir_path)

x1=['p0','p1','p2','p3','p4','p5','p6','p7']
exe_times = []
def myobj(point: dict):

  def plopper_func(x):
    x = np.asarray_chkfinite(x)  # ValueError if any NaN or Inf
    value = [point[x1[0]],point[x1[1]],point[x1[2]],point[x1[3]],point[x1[4]],point[x1[5]],point[x1[6]],point[x1[7]]] 
    print('VALUES:',point[x1[0]])
    params = ["P0","P1","P2","P3","P4","P5","P6","P7"]
    value = list(point.values())
    params = {k.upper(): v for k, v in point.items()}
    result = obj.findRuntime(value, params, ' -s large -m event -l 10000000') # defined(MINI_DATASET) && !defined(SMALL_DATASET) && !defined(MEDIUM_DATASET) && !defined(LARGE_DATASET) && !defined(EXTRALARGE_DATASET) && !defined(HUGE_DATASET)
    return result

  x = np.array(list(point.values())) #np.array([point[f'p{i}'] for i in range(len(point))])  
  results = plopper_func(x)
  exe_times.append(results)
  np.save(dir_path+'/exe_times.npy',exe_times)
#   results_s = sorted(results)
#   results_m = results_s[1:-1]
  print('OUTPUT:%f',results, float(np.mean(results[1:])))
  return float(np.mean(results[1:]))

In [ ]:
# name of csv file 
filename = "results_kde.csv"
fields   = ['p0','p1','p2','p3','p4','p5','p6','p7','exe_time','density']
# fields   = ['p1','p2','p3','p4','p5','exe_time','density']
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
#     csvwriter.writerows(rows)

    evals_infer = []
    for idx in range(N_infer):
        sample_point_val = new_kde[idx][0][-n_param:]
        print (sample_point_val)
        sample_point = {x1[0]:sample_point_val[0],
                    x1[1]:sample_point_val[1],
                    x1[2]:sample_point_val[2],
                    x1[3]:sample_point_val[3],
                    x1[4]:sample_point_val[4],
                    x1[5]:sample_point_val[5],
                    x1[6]:sample_point_val[6],
                    x1[7]:sample_point_val[7]}
#                     x1[8]:sample_point_val[8],
#                     x1[9]:sample_point_val[9]}
        res          = myobj(sample_point)
        print (sample_point, res)
        evals_infer.append(res)
        ss = [sample_point['p0']] + [sample_point['p1']] + [sample_point['p2']] + [sample_point['p3']] +[sample_point['p4']]+[sample_point['p5']]+[sample_point['p6']]+[sample_point['p7']]+[res]+[new_kde[idx][1]]
        csvwriter.writerow(ss)
        csvfile.flush()
csvfile.close()     

In [ ]:
x = [       1,     1000,    10000,   100000,  1000000,   5000000,  10000000,  15000000,  20000000]
y = [0.000001, 0.002513, 0.028749, 0.297755, 3.007381, 15.096215, 30.048476, 45.241969, 60.167836] 

In [ ]:
# import module
from tabulate import tabulate
  
# assign data
mydata = [{x[0], y[0]}, 
          {x[1], y[1]},
          {x[2], y[2]},
          {x[3], y[3]},
          {x[4], y[4]},
          {x[5], y[5]},
          {x[6], y[6]},
          {x[7], y[7]},
          {x[8], y[8]},
         ]
  
# create header
head = ["No. lookups", "execution time (sec.)"]
  
# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, y, 'ro')
plt.title('XSBench')
plt.xlabel('The number of lookups')
plt.ylabel('Execution time (sec.)')

In [ ]:
# x = [       1,     1000,    10000,   100000,   1000000,   5000000,   10000000,   15000000, 20000000]
x = [ '1','1,000','10,000','100,000','1,000,000','5,000,000','10,000,000','15,000,000','20,000,000']
y = [0.000002, 0.019164, 0.172359, 1.752702, 17.759856, 88.315073, 176.761855, 263.349105, 354.620147]

In [ ]:
x[3]

In [ ]:
# import module
from tabulate import tabulate
  
# assign data
mydata = [{x[0], y[0]}, 
          {x[1], y[1]},
          {x[2], y[2]},
          {x[3], y[3]},
          {x[4], y[4]},
          {x[5], y[5]},
          {x[6], y[6]},
          {x[7], y[7]},
          {x[8], y[8]},
         ]
  
# create header
head = ["No. lookups", "execution time (sec.)"]
  
# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, y, 'ro', label='XSBench')
plt.plot(x1, y1, 'bx', label='RSBench')
# plt.title('RSBench')
plt.legend()
plt.xlabel('The number of lookups')
plt.ylabel('Execution time (sec.)')

In [ ]:
6*60*60 
